DATA AUGMENTATION 

In [36]:
import pandas as pd
import shutil
import os
from PIL import Image
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import numpy as np
from collections import Counter
import shutil
import cv2
from typing_extensions import dataclass_transform

In [37]:
meta_data = pd.read_csv(r"C:\Users\vindu\Documents\FYP_NEW\ML_model\archive\HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [38]:
input_dir = r"C:\Users\vindu\Documents\FYP_NEW\ML_model\archive\Skin Cancer\Skin Cancer"
output_dir = r"C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\New_Dataset"

def is_rgb(image_path):
    try:
        with Image.open(image_path) as img:
            return img.mode == "RGB"
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return False

def filter_rgb_images(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(input_directory):
        file_path = os.path.join(input_directory, filename)

        if is_rgb(file_path):
            output_path = os.path.join(output_directory, filename)
            os.rename(file_path, output_path)
            print(f"Moved {filename} to {output_directory}")
        else:
            print(f"{filename} is not an RGB image.")

# Example usage

filter_rgb_images(input_dir, output_dir)


In [48]:
# Sorting images into appropriate directories in order
# to fascilitate real-time data augmentation while training.

# Using the cancer type label for directory creation.
dir_names = meta_data.dx.unique()

# Data Sorting process.
images_dir = r"C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\New_Dataset"
train_images_dir = r"C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\Train_Image"


def create_dirs(dir_path:str, dir_names:list):
    """
    This function creates directories within specified directory path
    with the provided list of directory names.

    Inputs
        dir_path:str - The path to which the new directories will reside in.
        dir_names:list - List name(s) of directories to be created.
    """
    # Looping through to create directories in new location.
    for dir_name in dir_names:
        try:
            os.makedirs(os.path.join(dir_path, str(dir_name)))
        except FileExistsError:
            continue

# Creating new directories.
create_dirs(train_images_dir, dir_names)

# Looping through each image in previous folder and
# assigning them to the appropriate folder
for image in os.scandir(images_dir):
    try:
        # attempting to rename image (moving to new dir).
        img_name = image.name.split(".")[0]
        img_cancer_type = str(meta_data.dx[meta_data.image_id == img_name].item()) # Retrieving the dx_label for image.
        shutil.copy(os.path.join(images_dir, image.name), os.path.join(train_images_dir, img_cancer_type, image.name))
    except Exception as e:
        print(e)

In [49]:
class SkinData:
  def __init__(self, root, transform = True):
      data_dirs = os.listdir(os.path.join(root,'Train_Image/'))
      self.data_files = []
      self.labels = []
      label = 0
      for dir in data_dirs:
        path_name = os.path.join(root,'Train_Image/',dir)
        data_samples = [os.path.join(path_name, fname) for fname in os.listdir(path_name)]
        label_samples = label * np.ones(len(data_samples))
        print(dir)
        print(len(label_samples))
        self.data_files.extend(data_samples)
        self.labels.extend(label_samples)
        self.transform = transform

        label += 1

  def __len__(self):
    return len(self.data_files)

  def __getitem__(self, index):
    new_width = 256
    new_height = 256
    data_file = self.data_files[index]
    image = np.array(cv2.imread(data_file))
    label = np.array(self.labels[index])

    data_sample = cv2.resize(image, (new_width, new_height))
    resize_data = np.array(data_sample)


    if self.transform:
      data_sample = torch.from_numpy(data_sample)
      lable = torch.from_numpy(label)



    data_point = {'data_sample': data_sample, 'lable':lable}


    return data_point

In [50]:
# Define your data directory and output directory
data_dir = r'C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\Train_Image'   # Folder where your dataset is stored (e.g., class folders)
output_dir = r'C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\Balanced_dataset'  # Folder where the balanced dataset will be saved
os.makedirs(output_dir, exist_ok=True)


In [54]:
import os
import shutil


def create_dirs(dir_path: str, dir_names: list):
    """
    This function creates directories within a specified directory path
    with the provided list of directory names.

    Inputs:
        dir_path: str - The path to which the new directories will reside in.
        dir_names: list - List name(s) of directories to be created.
    """
    for dir_name in dir_names:
        try:
            os.makedirs(os.path.join(dir_path, str(dir_name)))
        except FileExistsError:
            continue


# Define the directories
train_images_dir = r"C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\Train_Image"
validation_images_dir = r"C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\Validation_Image"

# Get the list of subdirectories (cancer types) from the train_images_dir
dir_names = [name for name in os.listdir(train_images_dir) if os.path.isdir(os.path.join(train_images_dir, name))]

inds = []  # list to contain directory names of each image.
five_percent_content = {}  # Dictionary containing cancer type and the value that is 5% of total number of that type of cancer images.

# Finding out how many images of each cancer type exist
for dir_name in os.scandir(train_images_dir):  # Iterating over all train images folders.
    for cancer_img in os.scandir(dir_name):    # Iterating over all images in all folders.
        inds.append(os.path.basename(os.path.dirname(cancer_img.path)))  # Appending directory name.

# Calculating number of specific type images & calculating what 5% of each image type will be.
for directory in dir_names:
    total_amt = inds.count(str(directory))
    print(f"There are {total_amt} images of {directory}.")
    print(f"5% of {directory} images is: {round(total_amt * 0.05, 0)}\n")
    five_percent_content[str(directory)] = round(total_amt * 0.05, 0)

# Creating and populating validation set directory
create_dirs(validation_images_dir, dir_names)

# Moving 5% of each type into its respective validation folder.
for sub_dir in os.scandir(train_images_dir):
    images_paths = [image.path for image in os.scandir(sub_dir)]
    for image_path in images_paths[:int(five_percent_content[str(sub_dir.name)])]:
        image_category = sub_dir.name
        shutil.move(image_path, os.path.join(validation_images_dir, image_category, os.path.basename(image_path)))


There are 327 images of akiec.
5% of akiec images is: 16.0

There are 514 images of bcc.
5% of bcc images is: 26.0

There are 1099 images of bkl.
5% of bkl images is: 55.0

There are 115 images of df.
5% of df images is: 6.0

There are 1113 images of mel.
5% of mel images is: 56.0

There are 6705 images of nv.
5% of nv images is: 335.0

There are 142 images of vasc.
5% of vasc images is: 7.0



In [55]:
# Define transformation for augmentation
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor()
])

# Define transformation for original images
original_transform = transforms.ToTensor()

# Load the dataset
dataset = datasets.ImageFolder(root=data_dir, transform=original_transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [58]:
# Count the number of images per class
class_counts = Counter([label for _, label in dataset.samples])
target_size = 1300 #max(class_counts.values())  # Set target size to the largest class size

In [59]:
class_counts

Counter({5: 6370, 4: 1057, 2: 1044, 1: 488, 0: 311, 6: 135, 3: 109})

In [60]:
# Function to save image to the appropriate class folder
def save_image(tensor, class_name, img_name, augment=False):
    img = transforms.ToPILImage()(tensor).convert("RGB")
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    prefix = "aug_" if augment else "orig_"
    img.save(os.path.join(class_dir, f"{prefix}{img_name}.jpeg"))

# Function to augment images and save
def augment_and_save_images(img, class_name, class_label, count_needed, j):
    for i in range(count_needed):
        augmented_img = augmentation_transforms(img)
        save_image(augmented_img, class_name, f"{j * count_needed - (count_needed - i+1)}", augment=True)

In [61]:
dataset.class_to_idx.items()

dict_items([('akiec', 0), ('bcc', 1), ('bkl', 2), ('df', 3), ('mel', 4), ('nv', 5), ('vasc', 6)])

In [62]:
# Balance the dataset
for class_name, class_label in dataset.class_to_idx.items():
    class_dir = os.path.join(data_dir, class_name)
    class_images = [img for img, label in dataset.samples if label == class_label]

    print(class_name)
    # print(len(class_images))

    # Current count of images in this class
    current_count = len(class_images)
    count_needed_original = target_size - current_count  # Calculate how many images are needed
    print(f"Augmenting class '{class_name}' with {count_needed_original} more images...")
    count_needed = round(count_needed_original/current_count)
    print(count_needed)

    # Augment images for this class if needed


    for i, img in enumerate(class_images):
        img = Image.open(img)
        original_img = original_transform(img)
        save_image(original_img, class_name, f"{class_label}_{i}", augment=False)
        if count_needed > 0:
          augment_and_save_images(img, class_name, class_label, count_needed, i+1)

        if i > target_size:
          break

akiec
Augmenting class 'akiec' with 989 more images...
3
bcc
Augmenting class 'bcc' with 812 more images...
2
bkl
Augmenting class 'bkl' with 256 more images...
0
df
Augmenting class 'df' with 1191 more images...
11
mel
Augmenting class 'mel' with 243 more images...
0
nv
Augmenting class 'nv' with -5070 more images...
-1
vasc
Augmenting class 'vasc' with 1165 more images...
9


In [63]:
dataset_2 = datasets.ImageFolder(root=output_dir, transform=original_transform)
class_counts = Counter([label for _, label in dataset_2.samples])

In [64]:
class_counts

Counter({1: 1464, 6: 1350, 3: 1308, 5: 1302, 0: 1244, 4: 1057, 2: 1044})

In [65]:
xx = Image.open(r'C:\Users\vindu\Documents\FYP_NEW\ML_model\Skin_Cancer\balanced_dataset\akiec\aug_1.jpeg')
xx.size

(600, 450)